In [ ]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Embedding, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers

# =====================
# 1. Setup determinístico
# =====================
os.environ['TF_DETERMINISTIC_OPS'] = '1'
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# =====================
# 2. Carregar datasets
# =====================
def load_dataset(path, sep='\t'):
    return pd.read_csv(path, sep=sep, encoding='utf-8')

X_train = load_dataset("data/test_input.csv")
y_train = load_dataset("data/test_output.csv")

X_val = load_dataset("data/human_ai_input.csv")
y_val = load_dataset("data/human_ai_output.csv")

X_test = load_dataset("data/dataset3_inputs.csv")
ids = X_test["ID"]

# =====================
# 3. Tokenização
# =====================
max_words = 20000
max_len = 500

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train["Text"])

def tokenize_pad(texts):
    seq = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seq, maxlen=max_len)

X_train_pad = tokenize_pad(X_train["Text"])
X_val_pad = tokenize_pad(X_val["Text"])
X_test_pad = tokenize_pad(X_test["Text"])

# =====================
# 4. Labels
# =====================
y_train = y_train["Label"].map({"AI": 1, "Human": 0}).values
y_val = y_val["Label"].map({"AI": 1, "Human": 0}).values

# =====================
# 5. Modelo MLP otimizado
# =====================
embedding_dim = 128
model = Sequential([
    Input((max_len,)),
    Embedding(max_words, embedding_dim, embeddings_initializer=initializers.GlorotUniform(seed=44)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# =====================
# 6. Treinar
# =====================
history = model.fit(X_train_pad, y_train, epochs=15, batch_size=64,
                    validation_data=(X_val_pad, y_val), verbose=1)

# =====================
# 7. Previsões
# =====================
preds = model.predict(X_test_pad)
pred_labels = ["AI" if p > 0.5 else "Human" for p in preds.flatten()]

# =====================
# 8. Exportar
# =====================
output_df = pd.DataFrame({
    "ID": ids,
    "Label": pred_labels
})
output_df.to_csv("data/previsao_melhorada.csv", sep='\t', index=False)
print("✅ Ficheiro de submissão gerado: previsao_melhorada.csv")


Epoch 1/15
47/47 [==============================] - 9s 124ms/step - loss: 0.4076 - accuracy: 0.8043 - val_loss: 0.0037 - val_accuracy: 0.9990
Epoch 2/15
47/47 [==============================] - 5s 98ms/step - loss: 0.0023 - accuracy: 0.9997 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 3/15
47/47 [==============================] - 5s 100ms/step - loss: 6.9537e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 4/15
47/47 [==============================] - 5s 115ms/step - loss: 2.6423e-04 - accuracy: 1.0000 - val_loss: 9.6524e-04 - val_accuracy: 1.0000
Epoch 5/15
47/47 [==============================] - 5s 97ms/step - loss: 1.9928e-04 - accuracy: 1.0000 - val_loss: 8.4273e-04 - val_accuracy: 1.0000
Epoch 6/15
47/47 [==============================] - 5s 97ms/step - loss: 1.9555e-04 - accuracy: 1.0000 - val_loss: 7.7413e-04 - val_accuracy: 1.0000
Epoch 7/15
47/47 [==============================] - 5s 108ms/step - loss: 1.0828e-04 - accuracy: 1.0000 - val_loss: 8.2007e